# The Human Metabolome Database (HMDB)

Author: Moshe Silverstein <br/>
Date: 11-17 <br/>
Data Source: http://www.hmdb.ca/

In [167]:
import sys, datetime, os
import numpy as np
import pandas as pd
import importlib
import untility_functions as uf
import xmltodict
%matplotlib inline

In [168]:
importlib.reload(uf)

<module 'untility_functions' from '/Users/moshesilverstein/Documents/Harmonizome/The Human Metabolome Database (HMDB)/untility_functions.py'>

# Load Data

In [170]:
with open('Input/hmdb_metabolites.xml') as fd:
    doc = xmltodict.parse(fd.read())

In [172]:
a = [dict(x) for x in dict(doc['hmdb'])['metabolite']]

In [173]:
df = pd.DataFrame.from_records(a)

In [174]:
df.head()

,abnormal_concentrations,accession,average_molecular_weight,bigg_id,biocyc_id,biofluid_locations,cas_registry_number,chebi_id,chemical_formula,chemspider_id,...,spectra,state,synonyms,synthesis_reference,taxonomy,tissue_locations,traditional_iupac,update_date,version,wikipidia
0,"{'concentration': [{'biofluid': 'Blood', 'conc...",HMDB0000001,169.1811,None,CPD-1823,"{'biofluid': ['Blood', 'Cerebrospinal Fluid (C...",332-80-9,50599,C7H11N3O2,83153,...,"{'spectrum': [{'type': 'Specdb::CMs', 'spectru...",Solid,"{'synonym': ['1 Methylhistidine', '1-Methyl hi...","Jain, Rahul; Cohen, Louis A. Regiospecific alk...",{'description': 'This compound belongs to the ...,"{'tissue': ['Muscle', 'Skeletal Muscle']}",1 methylhistidine,2017-08-16 03:09:05 UTC,3.6,None
1,"{'concentration': [{'biofluid': 'Blood', 'conc...",HMDB0000002,74.1249,36543,CPD-313,"{'biofluid': ['Blood', 'Feces', 'Urine']}",109-76-2,15725,C3H10N2,415,...,"{'spectrum': [{'type': 'Specdb::CMs', 'spectru...",Liquid,"{'synonym': ['1,3-Diamino-N-propane', '1,3-Pro...","Takayanagi, Yasuyuki; Oohinata, Takahiro. Pre...",{'description': 'This compound belongs to the ...,None,"α,ω-propanediamine",2017-08-16 03:09:07 UTC,3.6,None
2,"{'concentration': {'biofluid': 'Urine', 'conce...",HMDB0000005,102.0886,33889,2-OXOBUTANOATE,"{'biofluid': ['Blood', 'Cerebrospinal Fluid (C...",600-18-0,30831,C4H6O3,57,...,"{'spectrum': [{'type': 'Specdb::CMs', 'spectru...",Solid,"{'synonym': ['2-Ketobutanoate', '2-Ketobutanoi...","Figge, Rainer; Lux, Fabien; Raynaud, Celine; S...",{'description': 'This compound belongs to the ...,None,2-oxobutanoic acid,2017-08-16 03:09:09 UTC,3.6,Alpha-ketobutyric_acid
3,"{'concentration': [{'biofluid': 'Blood', 'conc...",HMDB0000008,104.1045,47130,CPD-3564,"{'biofluid': ['Blood', 'Cerebrospinal Fluid (C...",600-15-7,1148,C4H8O3,10792,...,"{'spectrum': [{'type': 'Specdb::CMs', 'spectru...",Solid,"{'synonym': ['(RS)-2-Hydroxybutyrate', '(RS)-2...","Carlier, J. P.; Henry, C.; Lorin, V.; Rouffign...",{'description': 'This compound belongs to the ...,{'tissue': 'Prostate'},α-hydroxybutyric acid,2017-08-16 03:09:11 UTC,3.6,2-Hydroxybutyric acid
4,"{'concentration': {'biofluid': 'Urine', 'conce...",HMDB0000010,300.3921,None,ESTRONE-SULFATE,"{'biofluid': ['Blood', 'Urine']}",362-08-3,1189,C19H24O3,389515,...,"{'spectrum': [{'type': 'Specdb::CMs', 'spectru...",Solid,"{'synonym': ['2-(8S,9S,13S,14S)-3-Hydroxy-2-me...","Stoelwinder, Johannes; Moers, Nicolaas Elisabe...",{'description': 'This compound belongs to the ...,None,2-methoxyestrone,2017-08-16 03:09:13 UTC,3.6,None


In [191]:
df.shape

(74507, 2)

In [189]:
df = df[['name', 'protein_associations']]

In [190]:
df.head()

,name,protein_associations
0,1-Methylhistidine,{'protein': [{'protein_accession': 'HMDBP00473...
1,"1,3-Diaminopropane",{'protein': [{'protein_accession': 'HMDBP00217...
2,2-Ketobutyric acid,{'protein': [{'protein_accession': 'HMDBP00012...
3,2-Hydroxybutyric acid,{'protein': [{'protein_accession': 'HMDBP00054...
4,2-Methoxyestrone,{'protein': [{'protein_accession': 'HMDBP00272...


In [241]:
df_interactions = pd.DataFrame()

for i, index in enumerate(df.index):
    
    lst2 = []
    
    progressPercent = ((i+1)/len(df.index))*100

    sys.stdout.write("Progeres: %d%%  %d Out of %d   \r" % (progressPercent, (i+1), len(df.index)))
    sys.stdout.flush()
    if type(df.ix[index, 'protein_associations']) != type(df.ix[114, 'protein_associations']):
        if type(df.ix[index, 'protein_associations']['protein']) == list:
            for j in range(0, len(df.ix[index, 'protein_associations']['protein'])):
                lst2.append(df.ix[index, 'protein_associations']['protein'][j]['gene_name'])
        else:
            lst2.append(df.ix[index, 'protein_associations']['protein']['gene_name'])
        lst1 = [df.ix[index, 'name']]*(len(lst2))
        temp = pd.DataFrame()
        temp['Metabolite'] = lst1
        temp['Gene'] = lst2
        df_interactions = pd.concat([df_interactions, temp]) 

In [242]:
df_interactions.head()

,Metabolite,Gene
0,1-Methylhistidine,CNDP1
1,1-Methylhistidine,PRMT3
0,"1,3-Diaminopropane",SMS
1,"1,3-Diaminopropane",DHPS
2,"1,3-Diaminopropane",ABP1


In [244]:
df_interactions.shape

(861526, 2)

In [245]:
df_interactions.reset_index(inplace=True)

In [246]:
df_interactions.drop('index', axis=1, inplace=True)

In [247]:
df_interactions.head()

,Metabolite,Gene
0,1-Methylhistidine,CNDP1
1,1-Methylhistidine,PRMT3
2,"1,3-Diaminopropane",SMS
3,"1,3-Diaminopropane",DHPS
4,"1,3-Diaminopropane",ABP1


In [248]:
df_interactions.drop_duplicates(inplace=True)

In [249]:
df_interactions.shape

(852388, 2)

# Map Gene Symbols To Up-to-date Approved Gene Symbols

In [250]:
df_interactions.set_index('Gene', inplace=True)

In [251]:
uf.mapgenesymbols(df_interactions)

# Drop Duplicates

In [252]:
df_interactions.reset_index(inplace=True)

In [253]:
df_interactions.drop_duplicates(inplace=True)

In [254]:
df_interactions.shape

(848829, 2)

# Create Binary Matrix

In [255]:
binary_matrix = uf.createBinaryMatix(df_interactions)

In [256]:
binary_matrix.head()

,"TG(18:3(9Z,12Z,15Z)/20:2n6/22:6(4Z,7Z,10Z,13Z,16Z,19Z))","TG(20:0/15:0/20:4(8Z,11Z,14Z,17Z))","TG(14:0/18:1(11Z)/18:3(9Z,12Z,15Z))",TG(16:1(9Z)/16:1(9Z)/20:1(11Z)),"PC(P-18:1(11Z)/20:3(5Z,8Z,11Z))",Folinic acid,"TG(20:1(11Z)/22:5(7Z,10Z,13Z,16Z,19Z)/22:5(4Z,7Z,10Z,13Z,16Z))","TG(22:0/22:4(7Z,10Z,13Z,16Z)/20:3n6)","TG(20:2n6/22:2(13Z,16Z)/20:4(8Z,11Z,14Z,17Z))",PE(16:0/14:1(9Z)),...,"TG(20:3(5Z,8Z,11Z)/22:0/20:4(5Z,8Z,11Z,14Z))","TG(18:1(9Z)/18:0/22:5(7Z,10Z,13Z,16Z,19Z))",TG(18:1(9Z)/24:0/20:2n6),"TG(16:0/14:0/18:2(9Z,12Z))","TG(20:0/18:3(6Z,9Z,12Z)/20:4(5Z,8Z,11Z,14Z))",TG(16:1(9Z)/18:1(11Z)/18:1(9Z)),"TG(16:0/24:1(15Z)/18:3(6Z,9Z,12Z))","TG(24:1(15Z)/22:4(7Z,10Z,13Z,16Z)/22:2(13Z,16Z))","PE(22:5(4Z,7Z,10Z,13Z,16Z)/22:2(13Z,16Z))",Flucytosine
AMPD3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
CDK4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ATP7B,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
SET,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
MERTK,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [257]:
binary_matrix.shape

(5309, 22128)

# Save Binary Matrix

In [258]:
filename = '~/./Documents/Harmonizome/The Human Metabolome Database (HMDB)/Output/hmdb_binary_matrix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
binary_matrix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene Set Library

In [259]:
path = '/Users/moshesilverstein/Documents/Harmonizome/The Human Metabolome Database (HMDB)/Output/'

In [260]:
name = 'hmdb_gene_set'

In [261]:
uf.createUpGeneSetLib(binary_matrix, path, name)

# Create Attribute Library

In [262]:
path = '/Users/moshesilverstein/Documents/Harmonizome/The Human Metabolome Database (HMDB)/Output/'

In [263]:
name = 'hmdb_attribute_set'

In [264]:
uf.createUpAttributeSetLib(binary_matrix, path, name)

# Create Gene Similarity Matrix

In [265]:
gene_similarity_matix = uf.createSimilarityMatrix(binary_matrix, 'jaccard')

In [266]:
gene_similarity_matix.head()

,AMPD3,CDK4,ATP7B,SET,MERTK,LPAR5,B3GAT2,LAT,TSTA3,SCGB1A1,...,MT1L,NME4,CATSPER1,CDKL5,NT5C2,XCR1,GTPBP10,NSD2,GSPT2,ARHGEF12
AMPD3,1.0,0.000000,0.100000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.103448,0.0,0.0,0.0,0.0,0.0
CDK4,0.0,1.000000,0.333333,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.086957,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0
ATP7B,0.1,0.333333,1.000000,0.0,0.333333,0.0,0.0,0.0,0.0,0.0,...,0.0,0.074074,0.0,0.333333,0.031250,0.0,0.0,0.0,0.0,0.0
SET,0.0,0.000000,0.000000,1.0,0.000000,0.0,0.0,1.0,0.0,0.0,...,0.0,0.000000,1.0,0.000000,0.000000,1.0,0.0,0.0,0.0,0.0
MERTK,0.0,1.000000,0.333333,0.0,1.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.086957,0.0,1.000000,0.000000,0.0,0.0,0.0,0.0,0.0


# Save Gene Similarity Matrix

In [267]:
filename = '~/./Documents/Harmonizome/The Human Metabolome Database (HMDB)/Output/hmdb_gene_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# Create Attribute Similarity matrix

In [268]:
attribute_similarity_matix = uf.createSimilarityMatrix(binary_matrix.T, 'jaccard')

In [269]:
attribute_similarity_matix.head()

,"TG(18:3(9Z,12Z,15Z)/20:2n6/22:6(4Z,7Z,10Z,13Z,16Z,19Z))","TG(20:0/15:0/20:4(8Z,11Z,14Z,17Z))","TG(14:0/18:1(11Z)/18:3(9Z,12Z,15Z))",TG(16:1(9Z)/16:1(9Z)/20:1(11Z)),"PC(P-18:1(11Z)/20:3(5Z,8Z,11Z))",Folinic acid,"TG(20:1(11Z)/22:5(7Z,10Z,13Z,16Z,19Z)/22:5(4Z,7Z,10Z,13Z,16Z))","TG(22:0/22:4(7Z,10Z,13Z,16Z)/20:3n6)","TG(20:2n6/22:2(13Z,16Z)/20:4(8Z,11Z,14Z,17Z))",PE(16:0/14:1(9Z)),...,"TG(20:3(5Z,8Z,11Z)/22:0/20:4(5Z,8Z,11Z,14Z))","TG(18:1(9Z)/18:0/22:5(7Z,10Z,13Z,16Z,19Z))",TG(18:1(9Z)/24:0/20:2n6),"TG(16:0/14:0/18:2(9Z,12Z))","TG(20:0/18:3(6Z,9Z,12Z)/20:4(5Z,8Z,11Z,14Z))",TG(16:1(9Z)/18:1(11Z)/18:1(9Z)),"TG(16:0/24:1(15Z)/18:3(6Z,9Z,12Z))","TG(24:1(15Z)/22:4(7Z,10Z,13Z,16Z)/22:2(13Z,16Z))","PE(22:5(4Z,7Z,10Z,13Z,16Z)/22:2(13Z,16Z))",Flucytosine
"TG(18:3(9Z,12Z,15Z)/20:2n6/22:6(4Z,7Z,10Z,13Z,16Z,19Z))",1.000000,1.000000,1.000000,0.641026,0.029126,0.0,1.000000,1.000000,1.000000,0.000000,...,1.000000,1.000000,1.000000,0.538462,1.000000,1.000000,1.000000,1.000000,0.000000,0.0
"TG(20:0/15:0/20:4(8Z,11Z,14Z,17Z))",1.000000,1.000000,1.000000,0.641026,0.029126,0.0,1.000000,1.000000,1.000000,0.000000,...,1.000000,1.000000,1.000000,0.538462,1.000000,1.000000,1.000000,1.000000,0.000000,0.0
"TG(14:0/18:1(11Z)/18:3(9Z,12Z,15Z))",1.000000,1.000000,1.000000,0.641026,0.029126,0.0,1.000000,1.000000,1.000000,0.000000,...,1.000000,1.000000,1.000000,0.538462,1.000000,1.000000,1.000000,1.000000,0.000000,0.0
TG(16:1(9Z)/16:1(9Z)/20:1(11Z)),0.641026,0.641026,0.641026,1.000000,0.020000,0.0,0.641026,0.641026,0.641026,0.000000,...,0.641026,0.641026,0.641026,0.866667,0.641026,0.641026,0.641026,0.641026,0.000000,0.0
"PC(P-18:1(11Z)/20:3(5Z,8Z,11Z))",0.029126,0.029126,0.029126,0.020000,1.000000,0.0,0.029126,0.029126,0.029126,0.474359,...,0.029126,0.029126,0.029126,0.020833,0.029126,0.029126,0.029126,0.029126,0.474359,0.0


# Save Attribute Similarity Matrix

In [270]:
filename = '~/./Documents/Harmonizome/The Human Metabolome Database (HMDB)/Output/hmdb_attribute_similarity_matix_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_similarity_matix.to_csv(filename, sep='\t', compression='gzip')

# Create Gene List

In [271]:
gene_list = uf.createGeneList(binary_matrix)

In [272]:
gene_list.head()

,GeneSym,GeneID
0,AMPD3,272
1,CDK4,1019
2,ATP7B,540
3,SET,6418
4,MERTK,10461


In [273]:
gene_list.shape

(5309, 2)

# Save Gene List

In [274]:
filename = '~/./Documents/Harmonizome/The Human Metabolome Database (HMDB)/Output/hmdb_gene_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
gene_list.to_csv(filename, sep='\t', index=False, compression='gzip')

# Create Attribute List

In [275]:
attribute_list = uf.createAttributeList(binary_matrix)

In [276]:
attribute_list.head()

,Attributes
0,"TG(18:3(9Z,12Z,15Z)/20:2n6/22:6(4Z,7Z,10Z,13Z,..."
1,"TG(20:0/15:0/20:4(8Z,11Z,14Z,17Z))"
2,"TG(14:0/18:1(11Z)/18:3(9Z,12Z,15Z))"
3,TG(16:1(9Z)/16:1(9Z)/20:1(11Z))
4,"PC(P-18:1(11Z)/20:3(5Z,8Z,11Z))"


In [277]:
attribute_list.shape

(22128, 1)

# Save Attribute List

In [278]:
filename = '~/./Documents/Harmonizome/The Human Metabolome Database (HMDB)/Output/hmdb_attribute_list_%s.tsv.zip'% str(datetime.date.today())[0:7].replace('-', '_')
attribute_list.to_csv(filename, sep='\t', index=False, compression='gzip')

# Create Gene-Attribute Edge List

In [279]:
path = '/Users/moshesilverstein/Documents/Harmonizome/The Human Metabolome Database (HMDB)/Output/'

In [280]:
name = 'hmdb_gene_attribute_edge_list'

In [281]:
uf.createGeneAttributeEdgeList(binary_matrix, gene_list, path, name)

Progeres: 100%  22128 Out of 22128   

 The number of statisticaly relevent gene-attribute associations is: 848829
